In [ ]:
from transformers import pipeline, set_seed
from datasets import load_dataset , load_from_disk, DatasetDict
import torch
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import nltk
from tqdm import tqdm
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import os
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer,AutoModelForSequenceClassification , Trainer, TrainingArguments, DataCollatorWithPadding
from sklearn.model_selection import train_test_split
import torch

# Set the environment variable for debugging
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# Ensure the cache directory exists (optional)



# Load the dataset (replace 'your_dataset' with your actual dataset name)
dataset = load_dataset('sealuzh/app_reviews')

# Split the dataset into train and validation sets
train_testvalid = dataset['train'].train_test_split(test_size=0.1)
train_test = train_testvalid['train'].train_test_split(test_size=0.1)
dataset = DatasetDict({
    'train': train_test['train'],
    'test': train_testvalid['test'],
    'valid': train_test['test']
})

# Load a pre-trained tokenizer
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# Define a function to convert examples to features
def convert_examples_to_features(example_batch):
    # Tokenize the reviews
    input_encodings = tokenizer(example_batch['review'], max_length=128, truncation=True, padding='max_length')

    # Map the ratings to a range of [0, 4] (if they are initially [1, 5])
    labels = [rating - 1 for rating in example_batch['star']]  # Adjust the mapping logic as needed

    return {
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': labels
    }

# Apply the function to the dataset #im herererererere----------------?
tokenized_datasets = dataset.map(convert_examples_to_features, batched=True)

# Convert to PyTorch tensors
tokenized_datasets.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/233332 [00:00<?, ? examples/s]

Map:   0%|          | 0/28807 [00:00<?, ? examples/s]

Map:   0%|          | 0/25926 [00:00<?, ? examples/s]

In [ ]:
!pip install transformers[torch] accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 13.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylin

In [ ]:

# Prepare the training, validation, and test sets
train_dataset = tokenized_datasets['train']
valid_dataset = tokenized_datasets['valid']
test_dataset = tokenized_datasets['test']

# Verify labels
#def verify_labels(dataset, num_labels):
   # for example in dataset:
       # for label in example['labels']:
      #      assert 0 <= label < num_labels, f"Label {label} is out of range"

#verify_labels(train_dataset, num_labels=5)
#verify_labels(valid_dataset, num_labels=5)
#verify_labels(test_dataset, num_labels=5)

# Debugging: Print some examples and their tensor sizes
print(f"Train dataset sample: {train_dataset[0]}")
print(f"Input IDs shape: {train_dataset[0]['input_ids'].shape}")
print(f"Attention mask shape: {train_dataset[0]['attention_mask'].shape}")
print(f"Labels shape: {train_dataset[0]['labels'].shape}")
print(f"Input IDs data type: {train_dataset[0]['input_ids'].dtype}")
print(f"Attention mask data type: {train_dataset[0]['attention_mask'].dtype}")
print(f"Labels data type: {train_dataset[0]['labels'].dtype}")

# Load the pre-trained BERT model with the adjusted num_labels
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment', num_labels=5)

# Create the data collator
data_collator = DataCollatorWithPadding(tokenizer)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # number of training epochs
    per_device_train_batch_size=8,   # batch size for training
    per_device_eval_batch_size=8,    # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    evaluation_strategy='epoch'      # Evaluate every epoch
)

# Initialize the Trainer
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=valid_dataset,          # evaluation dataset
    data_collator=data_collator          # data collator
)

# Train the model
trainer.train()

# Evaluate the model on the validation set
validation_results = trainer.evaluate(eval_dataset=valid_dataset)
print("Validation results:", validation_results)

# Evaluate the model on the test set
test_results = trainer.evaluate(eval_dataset=test_dataset)
print("Test results:", test_results)


Train dataset sample: {'input_ids': tensor([  101, 10171, 10103, 11706, 24531, 10103, 35821, 26613, 10165, 10297,
        13094, 10203,   151, 10743,   112,   162, 12696, 11318, 22777,   119,
        11364, 10203, 14325,   151, 25165, 50644, 10103, 35821, 11868, 10197,
        13667, 11111, 10114, 20082, 10573,   119, 15976, 13144, 11838,   119,
        48551, 12050, 14666,   119,   102,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
